In [ ]:
!pip install beautifulsoup4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#parsing links

def extract_drug_text(link):
  link_html = request.urlopen(link).read().decode('utf8')
  new_soup = BeautifulSoup(link_html)

  interactions = new_soup.find('h2', id='interactions')
  if interactions == None:
    return

  next = interactions.find_next('ul')

  links = next.find_all('a')
  related_drug_links = []

  for link in links:
    link = 'https://www.drugs.com' + link['href']
    if 'drug-interactions' in link:
      return parse_drug_interaction_page(link)
    related_drug_links.append(link)

  return related_drug_links


In [ ]:
#for drugs that have the 'check interactions' page

def parse_drug_interaction_page(link):

  #jumping to the interactions page
  link = link.replace('.html', '-index.html')
  link_html = request.urlopen(link).read().decode('utf8')
  new_soup = BeautifulSoup(link_html)

  li_list = h_soup.find_all('li', class_='int_2')
  drug_list = []
  for li in li_list:
    link = li.find_all('a')
    drug_list.append(link['href'])

  li_list = h_soup.find_all('li', class_='int_3')
  for li in li_list:
    link = li.find_all('a')
    drug_list.append(link['href'])

  return drug_list

In [ ]:
#web scraping portion

from bs4 import BeautifulSoup
from urllib import request
import os
import json

url = 'https://www.drugs.com/drug_information.html'
html = request.urlopen(url).read().decode('utf8')

soup = BeautifulSoup(html)

letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '0-9']

webpages = []
for letter in letters:

  top_level_url = 'https://www.drugs.com/alpha/' + letter + '.html'
  top_level_html = request.urlopen(top_level_url).read().decode('utf8')
  top_level_soup = BeautifulSoup(top_level_html)

  letter_pairs_list = top_level_soup.find('nav', class_='ddc-paging ddc-mgb-2')
  if letter_pairs_list == None:
    continue

  webpages = letter_pairs_list.find_all('a')
  if webpages == None:
    continue

  dict_related_drug_links = {}

  for page in webpages:

    page_link = 'https://www.drugs.com' + page['href']
    print(f'doing page {page_link}:')

    html = request.urlopen(page_link).read().decode('utf8')
    h_soup = BeautifulSoup(html)

    drug_list = h_soup.find('ul', class_='ddc-list-column-2')
    if drug_list == None:
      continue

    links = drug_list.find_all('a')


    for link in links:

      link = 'https://www.drugs.com' + link['href']
      related_drug_links = extract_drug_text(link)
      dict_related_drug_links[link] = related_drug_links

    filename = '/content/drive/MyDrive/drug_interactions/' + page['href'].replace('/', '_') + '.json'

    with open(filename, 'w') as f:
      json.dump(dict_related_drug_links, f, indent=4)


doing page https://www.drugs.com/alpha/ab.html:
doing page https://www.drugs.com/alpha/ac.html:
doing page https://www.drugs.com/alpha/ad.html:


KeyboardInterrupt: 

In [ ]:
import os
import json

# merge json files to create database

path = '/content/drive/MyDrive/drug_interactions'
files_in_path = os.listdir(path)
files_in_path.remove('.ipynb_checkpoints')

files = []
for f in files_in_path:
  f =  '/content/drive/MyDrive/drug_interactions/' + f #have to include the path to avoid error
  with open(f, 'r') as file:
    files.append(json.load(file))   #add all the files in one place

new_path = '/content/drive/MyDrive/merged_drug_interactions_file/' + 'combined_JSONs.json' #create new json file in path

with open(new_path, 'w') as o:
  json.dump(files, o, indent=4)   #put all content from files in one place in new path

In [ ]:
#create pandas DB from our merged json file
import pandas as pd
import json

path = '/content/drive/MyDrive/merged_drug_interactions_file/combined_JSONs.json'
with open(path, 'r') as path:
    json_data = json.load(path)

drug_names = []
drugs_that_interact_w_drugname = []

#putting stuff from json into 2 columns for the df
for drug in json_data:   #each drug has the drug name and the interactions list
    for c1, c2 in drug.items():  #c1 = drug, c2 = interactions list
        drug_names.append(c1)   #add drug name to list of names (drug name column)
        if c2 is None:    #if no interactions, is None
            drugs_that_interact_w_drugname.append('') #empty string bc there's nothing in the list, avoid error w/ join
        else:
            drugs_that_interact_w_drugname.append(', '.join(c2))  #create comma separated list in column of drugs

#creating the df using the two filled columns we've created
drug_interactions_db = pd.DataFrame(columns=['drug name', 'list of drugs that interact'])
drug_interactions_db['drug name'] = drug_names
drug_interactions_db['list of drugs that interact'] = drugs_that_interact_w_drugname

In [ ]:
drug_interactions_db

In [ ]:
import os

# Define the directory path
directory = '/content/drive/MyDrive/drug_interactions/database'

# Create the directory if it doesn't exist
os.makedirs(directory, exist_ok=True)

# Define the full file path
file_path = os.path.join(directory, 'drug_interactions_db.csv')

# Save the DataFrame to the file
drug_interactions_db.to_csv(file_path)

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/drug_interactions/database/drug_interactions_db.csv', index_col=0)
df

,drug name,list of drugs that interact
0,https://www.drugs.com/cons/a-b-otic.html,NaN
1,https://www.drugs.com/mtm/abacavir.html,"https://www.drugs.com/mtm/riociguat.html, http..."
2,https://www.drugs.com/mtm/abacavir-and-lamivud...,NaN
3,https://www.drugs.com/pro/abacavir-and-lamivud...,NaN
4,https://www.drugs.com/pro/abacavir-oral-soluti...,NaN
...,...,...
89248,https://www.drugs.com/pro/zyvana.html,NaN
89249,https://www.drugs.com/zyvox.html,NaN
89250,https://www.drugs.com/cons/zyvox-linezolid-int...,NaN
89251,https://www.drugs.com/cons/zyvox-linezolid-ora...,NaN
